In [1]:
import pytesseract
from PIL import Image, ImageEnhance, ImageFilter
from pdf2image import convert_from_path
import glob
from natsort import natsorted

from opennyai import Pipeline
from opennyai.utils import Data
from opennyai.ner import get_unique_provision_count

import pandas as pd
from tqdm import tqdm
import json


import shutil
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Sample POCSO Judgement

In [ ]:
pdf = "sample.pdf"
pages = convert_from_path(pdf, 350)

i = 1
for page in pages:
    image_name = "Page_" + str(i) + ".jpg"  
    page.save(image_name, "JPEG")
    i = i+1   

In [ ]:
pics = glob.glob("*.jpg")
pics = natsorted(pics)

In [ ]:
judgement_text = ''
for pic in pics:
    text = pytesseract.image_to_string(Image.open(pic))
    #os.remove(pic)
    judgement_text = judgement_text + text

In [ ]:
#print(judgement_text)

In [ ]:
# you can also load your text files directly into this
judgement_text = open('sample.txt').read()

texts_to_process = [judgement_text]

In [ ]:
# create Data object for data  preprocessing before running ML models
data = Data(texts_to_process, preprocessing_nlp_model='en_core_web_trf')

In [ ]:
# If you have access to GPU then set this to True else False
use_gpu = False

In [ ]:
# Load NER model
pipeline = Pipeline(components=['NER'], use_gpu=use_gpu, verbose=True,
                   ner_model_name='en_legal_ner_trf',
                   ner_mini_batch_size=40000,
                   ner_do_sentence_level=True,
                   ner_do_postprocess=True,
                   ner_statute_shortforms_path='')

In [ ]:
#Apply NER pipeline over the data
results = pipeline(data)

In [ ]:
# Export the results to a json file
import json
with open('output.json','w') as f:
     json.dump(results,f,indent=4)

In [ ]:
ner_doc_1 = pipeline._ner_model_output[0]

In [ ]:
# Frequency count of all provisions in a judgement
provisions=get_unique_provision_count(ner_doc_1)
provisions

In [ ]:
# Frequency count of all statues in a judgement
from opennyai.ner import get_unique_statute_count
statutes=get_unique_statute_count(ner_doc_1)
statutes

In [ ]:
# Saving NER Results
from opennyai.ner import get_csv
get_csv(ner_doc_1,'sample_judgement1_NER.csv', save_path=r"/home/krishna/NLP_Justice/POCSO/sample_judgement1_NER.csv")

In [ ]:
# Frequency count of all precendents in a judgement
from opennyai.ner import get_unique_precedent_count
import opennyai.ner as InLegalNER

precedents=InLegalNER.get_unique_precedent_count(ner_doc_1)
precedents

In [ ]:
ner_doc_1 = pipeline._ner_model_output[0]
identified_entites = [(ent, ent.label_) for ent in ner_doc_1.ents]
identified_entites

In [ ]:
ner_doc_1.user_data['precedent_clusters']

In [ ]:
with open('sample.txt', 'w') as f:
    f.write(judgement_text)

# All judgements

In [ ]:
judgement_paths_assam = []
path ="Assam"
for root, dirs, files in os.walk(path):
    for file in files:
        if(file.endswith(".pdf")):
            judgement_paths_assam.append(os.path.join(root,file))

In [ ]:
judgements = []
for judgement in judgement_paths_assam:
    if ('judgment' in judgement.lower()):
        judgements.append(judgement)

In [ ]:
len(judgements)

In [ ]:
## Convert Judgment PDFs into txts
#converted_cases = []
for judgement in tqdm(judgements):
    case_id = judgement.split(r'/')[-1].split('_')[0]
    if case_id in converted_cases:
        continue
        
    folder_path = judgement.split(r'/')
    folder_path.pop()
    folder_path = r"/".join(folder_path)+"/"

    pages = convert_from_path(judgement, 350, output_folder='/home/krishna/NLP_Justice/POCSO/tmp')
    
    i = 1
    for page in pages:
        image_name = "Page_" + str(i) + ".jpg"  
        page.save(image_name, "JPEG")
        i = i+1
    shutil.rmtree('tmp')
    os.makedirs('tmp')
        
    pics = glob.glob("*.jpg")
    pics = natsorted(pics)
    
    judgement_text = ''
    for pic in pics:
        text = pytesseract.image_to_string(Image.open(pic))
        os.remove(pic)
        judgement_text = judgement_text + text
    
    with open(folder_path+case_id+'.txt', 'w') as f:
        f.write(judgement_text)
        
    converted_cases.append(case_id)

In [5]:
judgement_paths_assam = []
path ="Assam"
for root, dirs, files in os.walk(path):
    for file in files:
        if(file.endswith(".txt")):
            judgement_paths_assam.append(os.path.join(root,file))

In [9]:
case_dfs = []
case_dfs.append(pd.read_csv('POCSO.csv'))

In [12]:
case_dfs[0].case_id.nunique()

9

In [13]:
for judgement in tqdm(judgement_paths_assam[case_dfs[0].case_id.nunique():]):
    judgement_text = open(judgement).read()
    #Get Case_ID 
    case_id = judgement.split(r'/')[-1].split('.txt')[0]
    
    # load your text files directly into this
    texts_to_process = [judgement_text]
    # create Data object for data  preprocessing before running ML models
    data = Data(texts_to_process, preprocessing_nlp_model='en_core_web_trf')
    
    # Load NER model
    pipeline = Pipeline(components=['NER'], use_gpu=False, verbose=True,
                   ner_model_name='en_legal_ner_trf',
                   ner_mini_batch_size=40000,
                   ner_do_sentence_level=True,
                   ner_do_postprocess=True,
                   ner_statute_shortforms_path='')
    
    results = pipeline(data)
    ner_doc_1 = pipeline._ner_model_output[0]
    
    #Get Provisions recognised in the judgement
    provisions=get_unique_provision_count(ner_doc_1)
    provisions_df = pd.DataFrame(provisions.items())
    provisions_df.columns = ['provision_opennyai','freq']
    
    # Clean Provisions and Statute text
    provisions_df['provision_opennyai'] = provisions_df['provision_opennyai'].str.lower().replace('section', 's', regex=True)
    provisions_df = provisions_df.groupby('provision_opennyai')[['freq']].sum().reset_index()
    provisions_df['statute_opennyai'] = ''
    
    for index, row in provisions_df.iterrows():
        statute = row['provision_opennyai'].split('of',1)[-1].strip().title()
        statute = statute.replace('Protection Of Children From Sexual Offences','POCSO')    
        provision = row['provision_opennyai'].split('of',1)[0].strip().title()
        provision = provision.replace('As Well As',r'/')
        provisions_df.loc[index,'statute_opennyai'] = statute
        provisions_df.loc[index,'provision_opennyai'] = provision.replace('S.','').strip()
        provisions_df.loc[index,'provision_opennyai'] = provision.replace('S','').strip()
    
    #Concatenate all provisions of a statute 
    provisions_df = provisions_df.groupby(['statute_opennyai'])['provision_opennyai'].apply(list).reset_index()
    provisions_df['case_id'] = case_id
    provisions_df = provisions_df[['case_id', 'statute_opennyai', 'provision_opennyai']]
    
    # Get Provisions and Statute information from parsed meta data
    folder_path = judgement.split(r'/')
    folder_path.pop()
    case_metadata_json = r"/".join(folder_path)+"/"+case_id+"_parsed.json"
    with open(case_metadata_json) as file:
        data = json.load(file)
    
    sections_df = pd.DataFrame(data['acts'])
    sections_df.columns = ['statute_metadata','provision_metadata']
    sections_df['case_id_meta'] = case_id
    
    # Concatenate Opennyai data and metadata
    df = pd.concat([provisions_df,sections_df], axis=1)
    df = df.drop('case_id_meta',axis=1)
    case_dfs.append(df)
    pd.concat(case_dfs).to_csv('POCSO.csv',index=False)

  0%|                                                  | 0/1152 [00:00<?, ?it/s]

ℹ Pre-processing will happen on CPU!
ℹ Loading NER...
ℹ NER will run on CPU!
ℹ Processing documents with Legal NER!!!



  0%|                                        | 1/1152 [00:24<7:49:50, 24.49s/it]

ℹ Pre-processing will happen on CPU!
ℹ Loading NER...
ℹ NER will run on CPU!
ℹ Processing documents with Legal NER!!!



  0%|                                        | 2/1152 [00:33<4:50:35, 15.16s/it]

...

⚠ There was some issue while performing postprocessing for doc id
066ac9ca5d713fb0ce8ce8d06e022108563ad10b66eb46c960c6595b48b8321d.
Some of postprocessing info may be absent because of this in doc.
ℹ Pre-processing will happen on CPU!
ℹ Loading NER...
ℹ NER will run on CPU!
ℹ Processing documents with Legal NER!!!



  6%|██▎                                    | 69/1152 [24:51<6:48:04, 22.61s/it]

ℹ Pre-processing will happen on CPU!
ℹ Loading NER...
ℹ NER will run on CPU!
ℹ Processing documents with Legal NER!!!



  6%|██▎                                    | 69/1152 [25:01<6:32:42, 21.76s/it]


KeyboardInterrupt: 

In [ ]:
# Frequency count of all statues in a judgement
from opennyai.ner import get_unique_statute_count
statutes=get_unique_statute_count(ner_doc_1)
statutes

In [ ]:
# Frequency count of all provisions in a judgement
from opennyai.ner import get_unique_provision_count
provisions=get_unique_provision_count(ner_doc_1)
provisions

In [ ]:
provisions_df = pd.DataFrame(provisions.items())
provisions_df.columns = ['provision','freq']
provisions_df['provision'] = provisions_df['provision'].str.lower().replace('section', 's', regex=True)

In [ ]:
provisions_df = provisions_df.groupby('provision')[['freq']].sum().reset_index()
provisions_df['statute'] = ''

In [ ]:
for index, row in provisions_df.iterrows():
    statute = row['provision'].split('of',1)[-1].strip().title()
    statute = statute.replace('Protection Of Children From Sexual Offences','POCSO')
    
    provision = row['provision'].split('of',1)[0].strip().title()
    provision = provision.replace('As Well As',r'/')
    provisions_df.loc[index,'statute'] = statute
    provisions_df.loc[index,'provision'] = provision

In [ ]:
provisions_df

In [ ]:
provisions_df = provisions_df.groupby(['statute'])['provision'].apply(list).reset_index()
provisions_df

In [ ]:
provisions_df['case_id'] = case_id

In [ ]:
provisions_df

In [ ]:
l = judgment.split(r'/')
l.pop()
case_metadata_json = r"/".join(l)+"/"+case_id+"_parsed.json"

import json
with open(case_metadata_json) as file:
    data = json.load(file)

In [ ]:
sections_df = pd.DataFrame(data['acts'])
sections_df.columns = ['statute_meta','provision_meta']
sections_df['case_id_meta'] = case_id

In [ ]:
k = pd.concat([provisions_df,sections_df], axis=1)

In [ ]:
k.drop('case_id_meta',axis=1)

# Limitations
1. The judgements are converted from PDFs to txt formats. The PDFs often contains official stamps and other symbols, which when converted into txt become a dirty group of letters. This made a couple of sentences in each page incomprehensible. The recognition of entities could be affected in these sentences.